In [2]:
# Path to unzipped dataset
nld_nao_path = "/code/nld-nao/nld-nao-unzipped"

# Database build
dbfilename = "nld-nao.db"

# Dataset name - human trajectories
dataset_name = "nld-nao"

In [1]:
import nle.dataset as nld
from nle.nethack import tty_render

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns

from nle.dataset import db

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Build the database if it doesn't already exist 
if not nld.db.exists(dbfilename):
    nld.db.create(dbfilename)
    # Add NLD-NAO data, use the `add_altorg_directory`.
    nld.add_altorg_directory(nld_nao_path, "nld-nao", dbfilename)

# Connect the database and print out the number of games to check the build
db_conn = nld.db.connect(filename=dbfilename)
print(f"NLD-NAO Dataset has {nld.db.count_games('nld-nao', conn=db_conn)} games.")

NLD-NAO Dataset has 1511228 games.


In [4]:
# Sample Dataset
dataset = nld.TtyrecDataset(
    "nld-nao",
    batch_size=32,
    seq_length=32,
    dbfilename="nld-nao.db",
)

# The keys for each timestep in the sequence 
minibatch = next(iter(dataset))
minibatch.keys()

dict_keys(['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids'])

In [5]:
# Top three maximum number of games played by player who never ascend
max_non_ascended_sql = """
WITH NonAscendedGames AS (
    SELECT 
        name,
        COUNT(*) AS non_ascended_count
    FROM 
        games
    WHERE 
        death != 'ascended'
    GROUP BY 
        name
)
SELECT 
    name,
    non_ascended_count
FROM 
    NonAscendedGames
ORDER BY 
    non_ascended_count DESC
LIMIT 
    3
"""

# Convert to dataframe
max_non_ascend = pd.read_sql_query(max_non_ascended_sql, db_conn)
print(max_non_ascend)

       name  non_ascended_count
0   Player3               16019
1  Player15               15159
2  Player69               11314


In [6]:
# Define a large data set of ascended players and non-ascended players
random_sample = """
WITH LARGEGAME AS (
    SELECT 
        name
    FROM 
        games
    GROUP BY 
        name
    HAVING 
        COUNT(*) > 275
)

SELECT 
    *
FROM 
    games
WHERE 
    name IN (SELECT name FROM LARGEGAME)
ORDER BY 
    name, starttime;
"""

# Execute the query and load the data into a pandas DataFrame
sample = pd.read_sql_query(random_sample, db_conn)

sample = sample.sort_values(by=['name','starttime'])

# Convert Unix time columns to datetime
sample['starttime'] = pd.to_datetime(sample['starttime'], unit='s')
sample['endtime'] = pd.to_datetime(sample['endtime'], unit='s')
sample

,gameid,version,points,deathdnum,deathlev,maxlvl,hp,maxhp,deaths,deathdate,...,death,conduct,turns,achieve,realtime,starttime,endtime,gender0,align0,flags
0,4085412,3.4.3,2091,2,4,4,0,32,1,20100730,...,killed by a dwarf,0xfc8,1152,0x0,2308,2010-07-30 08:23:27,2010-07-30 09:02:23,Fem,Cha,-1
1,4086650,3.4.3,144,0,2,3,-17,14,1,20100731,...,"killed by a guard, while sleeping",0xfc8,299,0x0,851,2010-07-30 09:02:50,2010-07-31 17:39:29,Fem,Cha,-1
2,4086653,3.4.3,0,0,1,1,-1,14,1,20100731,...,killed by a water demon,0xfff,6,0x0,50,2010-07-31 17:39:41,2010-07-31 17:40:35,Fem,Cha,-1
3,4086664,3.4.3,1806,2,5,5,0,32,1,20100731,...,killed by a housecat,0xfc0,2226,0x0,2288,2010-07-31 17:40:43,2010-07-31 18:18:57,Fem,Cha,-1
4,4086668,3.4.3,190,0,1,2,0,14,1,20100731,...,killed by a fox,0xfc8,448,0x0,309,2010-07-31 18:19:02,2010-07-31 18:24:20,Fem,Cha,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754935,1902415,3.6.3,132,0,2,2,0,15,1,20200911,...,killed by a sewer rat,0xf88,404,0x0,168,2020-09-11 19:54:40,2020-09-11 19:57:29,Mal,Law,0x4
754936,1902434,3.6.3,4507,2,7,10,0,65,1,20200916,...,killed by a jaguar while reading a book,0xf80,4729,0x0,2573,2020-09-14 23:41:24,2020-09-16 00:28:49,Mal,Law,0x0
754937,1902489,3.6.3,42,0,1,1,0,15,1,20200929,...,killed by a bear trap,0xf8f,179,0x0,62,2020-09-29 17:14:36,2020-09-29 17:15:39,Mal,Law,0x4
754938,1903284,3.6.3,2333,2,8,8,0,58,1,20201022,...,killed by a killer bee,0xf80,2913,0x0,8501,2020-10-18 23:17:07,2020-10-22 19:18:21,Mal,Law,0x4


In [7]:

# Unique players in the sample
print(f"Number of unique players: {sample['name'].nunique()}")

# Players who never Ascend
ascended_players = sample[sample['death'] == 'ascended']['name'].unique()
all_players = sample['name'].unique()
never_ascended_players = set(all_players) - set(ascended_players)
num_never_ascended_players = len(never_ascended_players)
print(f"Number of players who never ascended: {num_never_ascended_players}")

# Therefore, roughly half of the players ascend at least once, and the rest do not


Number of unique players: 1012
Number of players who never ascended: 438


In [8]:
sample['points_trend'] = sample.groupby('name')['points'].transform(lambda x: np.polyfit(range(len(x)), x, 1)[0])
sample['maxlvl_trend'] = sample.groupby('name')['maxlvl'].transform(lambda x: np.polyfit(range(len(x)), x, 1)[0])

# Aggregate features by player
player_features = sample.groupby('name').agg({
    'points_trend': 'mean',
    'maxlvl_trend': 'mean',
    'points': ['mean', 'std'],
    'maxlvl': ['mean', 'std']
}).reset_index()

scaler = StandardScaler()
# Flatten the MultiIndex columns
player_features.columns = ['_'.join(col).strip() for col in player_features.columns.values]

# Now you can drop the 'name' column without warnings
scaled_features = scaler.fit_transform(player_features.drop('name_', axis=1))
player_features = pd.DataFrame(scaled_features)

kmeans = KMeans(n_clusters=5, random_state=42)
player_features['cluster'] = kmeans.fit_predict(scaled_features)



In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns to normalize
columns_to_normalize = ['points', 'maxlvl', 'turns', 'maxhp']

# Fit and transform the selected columns
sample[columns_to_normalize] = scaler.fit_transform(sample[columns_to_normalize])

# Display the normalized data
print(sample.head())

grouped = sample.groupby('name')

# Plot for each player
for name, group in grouped:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f"Player: {name}", fontsize=16)

    # Plot Points over Time
    axes[0, 0].plot(group['starttime'], group['points'], marker='o')
    axes[0, 0].set_title('Normalized Points Over Time')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('Normalized Points')

    # Plot Maxlvl over Time
    axes[0, 1].plot(group['starttime'], group['maxlvl'], marker='o', color='orange')
    axes[0, 1].set_title('Normalized Max Level Over Time')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Normalized Max Level')

    # Plot Turns over Time
    axes[1, 0].plot(group['starttime'], group['turns'], marker='o', color='green')
    axes[1, 0].set_title('Normalized Turns Over Time')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Normalized Turns')

    # Plot MaxHP over Time
    axes[1, 1].plot(group['starttime'], group['maxhp'], marker='o', color='red')
    axes[1, 1].set_title('Normalized Max HP Over Time')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Normalized Max HP')

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [18]:
# Consider trajectories of point, turns and levels reached 


In [ ]:
# Rolling Window over data frame to obtain insight about frequency of gameplay 
import pandas as pd
import matplotlib.pyplot as plt

# Function to find high-frequency gameplay periods
def find_high_frequency_periods(df, window_size='30D', threshold=10):
    """
    Identify periods of high gameplay frequency for a player.
    
    Parameters:
    - df: DataFrame with player's data containing 'starttime'
    - window_size: Rolling window size (e.g., '30D' for 30 days)
    - threshold: Minimum number of games in the window to consider it a high-frequency period
    
    Returns:
    - List of time periods with high gameplay frequency
    """
    # Resample or calculate rolling count of games
    df['game_count'] = df['starttime'].dt.to_period('D')  # Convert to daily periods
    rolling_counts = df.set_index('starttime').resample(window_size)['game_count'].count()  # Rolling window count

    # Identify high-frequency periods
    high_freq_periods = rolling_counts[rolling_counts >= threshold].index
    return high_freq_periods

# Group Player by name and iterate through this 
grouped = sample.groupby('name')
high_frequency_periods = {}


for name, group in grouped:
    periods = find_high_frequency_periods(group)
    high_frequency_periods[name] = periods
    print(f"Player: {name}, High-Frequency Periods: {periods}")

# Example of visualization for a specific player
player_name = 'Player10012'  # Example player from the image
player_data = grouped.get_group(player_name)
high_periods = find_high_frequency_periods(player_data)

plt.figure(figsize=(12, 6))
plt.plot(player_data['starttime'], player_data['points'], 'o', alpha=0.5, label='Gameplay')
for period in high_periods:
    plt.axvspan(period.start_time, period.end_time, color='yellow', alpha=0.3)  # Highlight periods
plt.title(f"High Frequency Gameplay Periods for {player_name}")
plt.xlabel('Time')
plt.ylabel('Points')
plt.legend()
plt.show()

In [ ]:
data = data[0:1000]
grouped = data.groupby('name')

# Plot for each player
for name, group in grouped:
    fig, axes = plt.subplots(1, 2, figsize=(14, 10))
    fig.suptitle(f"Player: {name}", fontsize=16)

    # Plot Points over Time
    axes[0].plot(group['starttime'], group['points'], marker='o')
    axes[0].set_title('Normalized Points Over Time')
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('Normalized Points')

    # Plot Maxlvl over Time
    axes[1].plot(group['starttime'], group['maxlvl'], marker='o', color='orange')
    axes[1].set_title('Normalized Max Level Over Time')
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Normalized Max Level')

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [45]:
# Trend Analysis
maxlvl_increase_sql = """
WITH LevelChange AS (
    SELECT 
        name,
        gameid,
        maxlvl,
        starttime,
        LEAD(maxlvl) OVER (PARTITION BY name ORDER BY starttime) - maxlvl AS level_diff
    FROM games
),

UpwardTrend AS (
    SELECT
        name,
        COUNT(CASE WHEN level_diff > 0 THEN 1 END) AS positive_diff_count,
        COUNT(CASE WHEN level_diff < 0 THEN 1 END) AS negative_diff_count,
        COUNT(*) AS total_games
    FROM LevelChange
    GROUP BY name
    HAVING COUNT(CASE WHEN level_diff > 0 THEN 1 END) >= (COUNT(*) * 0.65)  -- Players with an upward trend in at least 80% of their games
)

SELECT 
    g.*
FROM
    games g
JOIN
    UpwardTrend ut ON g.name = ut.name
ORDER BY
    g.name, g.starttime
"""

# Execute the query and load the data into a pandas DataFrame
data = pd.read_sql_query(maxlvl_increase_sql, db_conn)
data

,gameid,version,points,deathdnum,deathlev,maxlvl,hp,maxhp,deaths,deathdate,...,death,conduct,turns,achieve,realtime,starttime,endtime,gender0,align0,flags
0,1905856,3.6.1,89,0,1,1,12,12,0,20161111,...,escaped,0xfcf,170,0x0,8228,1478825086,1478834428,Fem,Neu,0x4
1,1905857,3.6.1,166,0,2,2,0,13,1,20161111,...,killed by a jackal,0xfcf,476,0x0,610,1478834625,1478835240,Fem,Cha,0x4
2,1905858,3.6.1,655,2,3,3,-1,25,1,20161111,...,killed by a crossbow bolt,0xfdf,605,0x0,888,1478835301,1478836259,Mal,Cha,0x4
3,40057,3.6.4,5,0,1,1,12,12,0,20200124,...,quit,0xfff,75,0x0,7758,1579782302,1579896280,Mal,Cha,0x4
4,40060,3.6.4,89,0,2,2,0,12,1,20200124,...,killed by a goblin,0xfc8,365,0x0,307,1579896301,1579896609,Mal,Cha,0x4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,1871049,3.6.3,211,0,2,3,19,19,0,20191101,...,quit,0xf88,1089,0x0,1734,1572575408,1572577145,Mal,Neu,0x4
1584,1871076,3.6.3,2293,2,5,6,0,38,1,20191101,...,killed by a gnome mummy while praying,0xf80,2771,0x0,5424,1572577190,1572582689,Mal,Neu,0x4
1585,1871086,3.6.3,666,0,7,7,0,22,1,20191101,...,killed by a homunculus,0xf88,836,0x0,849,1572583272,1572584123,Mal,Neu,0x4
1586,1871088,3.6.3,283,0,5,5,0,15,1,20191101,...,killed by a water moccasin,0xfc8,527,0x0,503,1572584213,1572584717,Mal,Neu,0x4


In [ ]:
# Clustering with DBSCAN 
# Handles nested clusters 
# Will do three-dimensional clustering 
# DBSCAN: Define radius for point-wise iteration over maximum neighbors 
# Define core points and extend cluster through iteration with radius membership based on core cluster
# Add non-core points to cluster closest to other core points, but don't add non-core points to non-core point clusters 

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Function to perform DBSCAN clustering on maxlvl, points, and turns
def find_gameplay_clusters(player_data, eps=0.5, min_samples=5):
    """
    Perform DBSCAN clustering on the gameplay metrics maxlvl, points, and turns.

    Parameters:
    - player_data: DataFrame with a player's gameplay data containing 'maxlvl', 'points', and 'turns'
    - eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
    - min_samples: Minimum number of samples in a cluster for it to be considered a valid cluster
    
    Returns:
    - player_data: The DataFrame with an additional column 'cluster' indicating cluster membership
    """
    # Select features to cluster on and scale them
    features = player_data[['maxlvl', 'points', 'turns']]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Apply DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(scaled_features)
    
    # Assign cluster labels to the data
    player_data['cluster'] = db.labels_
    
    return player_data

# Group data by player and apply clustering
grouped = data.groupby('name')
high_frequency_clusters = {}

for name, group in grouped:
    clusters = find_gameplay_clusters(group)
    high_frequency_clusters[name] = clusters
    print(f"Player: {name}, High-Frequency Clusters:\n", clusters)

# Group data by player and apply clustering
grouped = data.groupby('name')
all_clusters = []

for name, group in grouped:
    clusters = find_gameplay_clusters(group)
    all_clusters.append(clusters)


In [36]:
# Concatenate all clustered data
clustered_data = pd.concat(all_clusters)

# Plotting gameplay clusters
def plot_clusters(clustered_data, player_name):
    # Filter data for the specific player
    player_data = clustered_data[clustered_data['name'] == player_name]

    # Define cluster colors (noise points are -1)
    colors = plt.cm.viridis(np.linspace(0, 1, len(player_data['cluster'].unique())))

    # Create scatter plots for each combination of features
    plt.figure(figsize=(18, 6))

    # Plot maxlvl vs points
    plt.subplot(1, 3, 1)
    scatter = plt.scatter(
        player_data['maxlvl'], 
        player_data['points'], 
        c=player_data['cluster'], 
        cmap='viridis', 
        alpha=0.7
    )
    plt.xlabel('Max Level')
    plt.ylabel('Points')
    plt.title(f'Clusters of Max Level vs Points for {player_name}')

    # Plot maxlvl vs turns
    plt.subplot(1, 3, 2)
    plt.scatter(
        player_data['maxlvl'], 
        player_data['turns'], 
        c=player_data['cluster'], 
        cmap='viridis', 
        alpha=0.7
    )
    plt.xlabel('Max Level')
    plt.ylabel('Turns')
    plt.title(f'Clusters of Max Level vs Turns for {player_name}')

    # Plot points vs turns
    plt.subplot(1, 3, 3)
    plt.scatter(
        player_data['points'], 
        player_data['turns'], 
        c=player_data['cluster'], 
        cmap='viridis', 
        alpha=0.7
    )
    plt.xlabel('Points')
    plt.ylabel('Turns')
    plt.title(f'Clusters of Points vs Turns for {player_name}')

    # Add a colorbar to show the cluster numbers
    plt.colorbar(scatter, ax=plt.gcf().get_axes(), label='Cluster')
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Sample data preparation
# Assuming 'data' is a DataFrame with columns 'name', 'maxlvl', 'points', 'turns', and 'starttime'
# Convert 'starttime' to datetime if not already done
sample['starttime'] = pd.to_datetime(sample['starttime'], unit='s')  # Adjust the unit if necessary

# Function to perform DBSCAN clustering on maxlvl, points, and turns
def find_gameplay_clusters(player_data, eps=0.5, min_samples=5):
    # Check if the data is empty
    if player_data.empty:
        print("No data available for clustering.")
        return player_data
    
    # Select features to cluster on and scale them
    features = player_data[['maxlvl', 'points', 'turns']].dropna()  # Drop rows with NaN values
    
    # Check if there are enough samples after filtering
    if features.empty:
        print("Insufficient data after filtering to perform clustering.")
        player_data['cluster'] = -1  # Label all as noise
        return player_data
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Apply DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(scaled_features)
    
    # Assign cluster labels to the data
    player_data = player_data.loc[features.index]  # Align player_data with scaled features
    player_data['cluster'] = db.labels_
    
    return player_data

# List of players to generate plots for
player_names = sample['name'].unique()  # Use this to get a list of all players or a subset

# Iterate over each player and generate plots
for player_name in player_names:
    player_data = sample[sample['name'] == player_name]

    # Check if the player data is empty
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        continue
    
    # Apply clustering
    clustered_data = find_gameplay_clusters(player_data, eps=0.5, min_samples=5)

    # Plotting the gameplay clusters
    if 'cluster' in clustered_data.columns and not clustered_data.empty:
        plt.figure(figsize=(14, 6))

        # Scatter plot for maxlvl vs points
        plt.subplot(1, 2, 1)
        scatter = plt.scatter(
            clustered_data['maxlvl'], 
            clustered_data['points'], 
            c=clustered_data['cluster'], 
            cmap='viridis', 
            alpha=0.7, 
            edgecolors='w'
        )
        plt.title(f"Max Level vs Points Clustering for {player_name}")
        plt.xlabel('Max Level')
        plt.ylabel('Points')
        plt.colorbar(scatter, label='Cluster')

        # Scatter plot for turns vs points
        plt.subplot(1, 2, 2)
        scatter = plt.scatter(
            clustered_data['turns'], 
            clustered_data['points'], 
            c=clustered_data['cluster'], 
            cmap='viridis', 
            alpha=0.7, 
            edgecolors='w'
        )
        plt.title(f"Turns vs Points Clustering for {player_name}")
        plt.xlabel('Turns')
        plt.ylabel('Points')
        plt.colorbar(scatter, label='Cluster')

        plt.tight_layout()
        plt.show()
    else:
        print(f"No valid clusters found to plot for {player_name}.")



In [ ]:
# Clustered Sampling with SQL 
# Consider Strata for Defining a homogenous subsample of the data, then perform clustering methods on sample 
#

In [ ]:
# Cluster Analysis - Understand characteristics of each group
# Use Cluster Sampling to identify the characteristics of interest from NLD-NAO

In [12]:
# Scatterplot or histogram will provide better insight over clustering methods 
# Clusters may be arbitrary threshold splits of a continuous value
# Small fraction of power users
# Look at lots of histograms of everything you can imagine 
# Preserve the ability to run old experiments 
# Need to answer what is inadequate about the dataset
# Go for a simple definition
# Look at early games and later games 
# Simple thresholding to define a simple gap 
# Measure of progress, look at forecasting model to see if a player has a high likelihood of ascension based on some number of games
# Get something where it is good enough so we can find where it is not good enough
# Big simple patterns that capture most things. For example, never ascend but have a game that is very high level then you are more likely to eventually ascend
# Find obvious trends, set up environment so you can iterate quickly
# Finding signal for fitting a model 
# Organize the Clutter and improve iteration process 
# Python files that represent function library 
# Good goal for Friday: Model for predicting ascension or some measure of success with at least 1000 players 
# Analysis of how effective individual features are for that prediction 
# Which individual features are measurable progress for persistence in NetHack 

# SQL query to retrieve all rows for players who played at least 300 games and ascended
query = """
WITH PlayerGameCounts AS (
    SELECT 
        name,
        COUNT(*) AS games_played
    FROM 
        games
    GROUP BY 
        name
    HAVING 
        COUNT(*) >= 300
),
AscendedPlayers AS (
    SELECT 
        DISTINCT name
    FROM 
        games
    WHERE 
        death = 'ascended'
),
EligiblePlayers AS (
    SELECT 
        p.name
    FROM 
        PlayerGameCounts p
    JOIN 
        AscendedPlayers a ON p.name = a.name
)
SELECT 
    g.*
FROM 
    games g
JOIN 
    EligiblePlayers ep ON g.name = ep.name
ORDER BY 
    g.name, g.starttime;
"""

# Execute the query
all_rows_for_players = pd.read_sql_query(query, db_conn)



# Display the result
all_rows_for_players

,gameid,version,points,deathdnum,deathlev,maxlvl,hp,maxhp,deaths,deathdate,...,death,conduct,turns,achieve,realtime,starttime,endtime,gender0,align0,flags
0,2908240,3.6.0,398,0,3,3,0,22,1,20180402,...,killed by a gecko,0xf8c,1773,0x0,16712,1522676195,1522682518,Fem,Neu,0x4
1,2908275,3.6.0,166,0,2,2,-2,11,1,20180402,...,killed by a sewer rat,0xfc8,1147,0x0,968,1522683949,1522684499,Fem,Cha,0x4
2,2908310,3.6.0,696,0,3,3,-1,40,1,20180402,...,killed by a wererat while frozen by a monster'...,0xf88,2243,0x0,2382,1522684596,1522685763,Mal,Cha,0x4
3,2908471,3.6.0,562,0,3,3,-3,33,1,20180402,...,killed by a wand,0xf88,1743,0x0,3317,1522696591,1522698420,Mal,Neu,0x4
4,2908476,3.6.0,932,0,3,3,0,35,1,20180402,...,killed by a coyote while frozen by a monster's...,0xf88,2592,0x0,2501,1522698490,1522699738,Fem,Neu,0x4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405664,6536031,3.4.3,2912,2,4,4,-6,37,1,20171028,...,killed by a dwarf,0xf80,2038,0x0,698,1509213708,1509214406,Mal,Cha,0x0
405665,6536204,3.4.3,474,0,2,2,-4,27,1,20171030,...,killed by a bolt of lightning,0xfc8,1003,0x0,393,1509361268,1509361663,Mal,Cha,0x0
405666,6536205,3.4.3,93,0,1,1,0,11,1,20171030,...,killed by a fox,0xfc8,206,0x0,75,1509361666,1509361741,Mal,Cha,0x0
405667,2782906,3.6.0,90,0,1,1,8,11,0,20171102,...,quit,0xfcf,125,0x0,200,1509615957,1509616158,Mal,Cha,0x4
